In [1]:
import os
import numpy as np
import pandas as pd
import pickle
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [2]:
def read_excel(path):
    return pd.read_excel(path)

def perhitungan_jumlah_smp(df):
    df['SMP_MTR'] = df['MOTOR'] * 0.8
    df['SMP_MBL'] = df['MOBIL']
    df['SMP_TRK'] = df['TRUK/BUS'] * 2.5
    df['SMP'] = round(df['SMP_MTR'] + df['SMP_MBL'] + df['SMP_TRK'])
    df['JUMLAH'] = df['MOTOR'] + df['MOBIL'] + df['TRUK/BUS']
    return df

def preprocessing(df):
    encoder = OneHotEncoder(sparse_output=False)
    scaled_features = StandardScaler()
    
    # Encode HARI (pastikan encoder sudah fit pada data sebelumnya atau gunakan yang sudah disimpan)
    encoded_days = encoder.fit_transform(df[['HARI']])
    
    # Konversi 'JAM' ke total menit sejak tengah malam
    df['JAM'] = df['JAM'].apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1]))

    # Gabungkan encoded 'HARI' dengan fitur 'JAM'
    X_missing = pd.DataFrame(encoded_days, columns=encoder.get_feature_names_out(['HARI']))
    X_missing['JAM'] = scaled_features.fit_transform(df[['JAM']])

    # Tambahkan kolom Bulan
    X_missing.insert(0, 'BULAN', df['BULAN'])

    with open('models/model_rf_mtr_bolanggandu_2022.pkl', 'rb') as file:
        model_rf_mtr = pickle.load(file)

    # Memuat model dari file
    with open('models/model_rf_mbl_bolanggandu_2022.pkl', 'rb') as file:
        model_rf_mbl = pickle.load(file)

    # Memuat model dari file
    with open('models/model_rf_bus_bolanggandu_2022.pkl', 'rb') as file:
        model_rf_bus = pickle.load(file)

    # Melakukan prediksi untuk kolom MOTOR
    motor_predictions = model_rf_mtr.predict(X_missing)
    mobil_predictions = model_rf_mbl.predict(X_missing)
    bus_predictions = model_rf_bus.predict(X_missing)

    return motor_predictions, mobil_predictions, bus_predictions
    

def minutes_to_time(minutes):
    hours = minutes // 60
    minutes = minutes % 60
    return f'{int(hours):02d}:{int(minutes):02d}'

def menggabungkan_data(df, missing_times):
    # Menggabungkan data hilang dengan DataFrame utama
    df = pd.concat([df, missing_times], ignore_index=True)
    
    # Mengurutkan DataFrame berdasarkan kolom 'Waktu'
    df = df.sort_values(by=['TAHUN','BULAN','TANGGAL','HARI','JAM']).reset_index(drop=True)

    return df


In [3]:
# path_1 = '../Data/Preprocessing/Data Balonggandu/2023/1_input_balonggandu_2023.xlsx'
path_2 = '../Data/Preprocessing/Data Balonggandu/2022/1_missing_times_balonggandu_2022.xlsx'

# df = read_excel(path_1)
df_missing = read_excel(path_2)

print(len(df_missing))

35040


In [4]:
df_missing.head()

,WAKTU,TAHUN,BULAN,PEKAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2022-01-01 00:00:00,2022,1,1,1,Saturday,00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-01 00:15:00,2022,1,1,1,Saturday,00:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-01 00:30:00,2022,1,1,1,Saturday,00:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-01 00:45:00,2022,1,1,1,Saturday,00:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-01 01:00:00,2022,1,1,1,Saturday,01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
motor_predictions, mobil_predictions, bus_predictions = preprocessing(df_missing)

In [6]:
# Menambahkan prediksi ke dalam dataframe
df_missing['MOTOR'] = motor_predictions.round()
df_missing['MOBIL'] = mobil_predictions.round()
df_missing['TRUK/BUS'] = bus_predictions.round()

In [7]:
df_missing = perhitungan_jumlah_smp(df_missing)
df_missing['JAM'] = df_missing['JAM'].apply(minutes_to_time)

In [8]:
df_missing.head()

,WAKTU,TAHUN,BULAN,PEKAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2022-01-01 00:00:00,2022,1,1,1,Saturday,00:00,58.0,40.0,36.0,134.0,46.4,40.0,90.0,176.0
1,2022-01-01 00:15:00,2022,1,1,1,Saturday,00:15,54.0,43.0,37.0,134.0,43.2,43.0,92.5,179.0
2,2022-01-01 00:30:00,2022,1,1,1,Saturday,00:30,37.0,41.0,35.0,113.0,29.6,41.0,87.5,158.0
3,2022-01-01 00:45:00,2022,1,1,1,Saturday,00:45,39.0,42.0,28.0,109.0,31.2,42.0,70.0,143.0
4,2022-01-01 01:00:00,2022,1,1,1,Saturday,01:00,35.0,45.0,41.0,121.0,28.0,45.0,102.5,176.0


In [9]:
df_compiled = df_missing.copy()

In [10]:
df_missing.to_excel('../Data/Preprocessing/Data Balonggandu/2022/2_missing_times_balonggandu_2022.xlsx', index=False)
df_compiled.to_excel('../Data/Preprocessing/Data Balonggandu/2022/2_input_balonggandu_2022.xlsx', index=False)